In [1]:
import json
import os
import ase.io

from ase.io               import read,write
from ase.io               import vasp
from hiphive              import ClusterSpace, StructureContainer, ForceConstantPotential
from hiphive.utilities    import prepare_structures,get_displacements
from trainstation         import Optimizer
from ase.calculators.vasp import Vasp
from ase.io.vasp          import read_vasp,write_vasp




Read structures containing displacements and forces

In [2]:
volumes   = ['75.18', '75.56', '75.93', '76.31', '76.69']
root_path = f'data/CrN-riccardo/SC_2x2x2_Vol_{volumes[0]}'

# Create and save as a dictionary
setup_variables = {
    'root_path':                   root_path,
    'path_to_relaxations':         f'{root_path}/calculations',
    'path_to_primtive_cell':       f'{root_path}/POSCAR_unit',
    'path_to_super_cell':          f'{root_path}/POSCAR_supercell',
    'path_to_cluster_space':       f'{root_path}/cluster_space.cs',
    'path_to_structure_container': f'{root_path}/structure_container.sc',
    'path_to_fcp':                 f'{root_path}/material.fcp'
}

# Write the dictionary to the file in JSON format
with open(f'setup_variables.json', 'w') as json_file:
    json.dump(setup_variables, json_file)

In [3]:
# Cutoffs for pairs, triplets and quadruplets (in angstrom) 
cutoffs = [4.0, 4.0, 3.0]

In [4]:
primtive_cell = read_vasp(setup_variables['path_to_primtive_cell'])
super_cell = read_vasp(setup_variables['path_to_super_cell'])

rattled_structures = []
for folder in os.listdir(setup_variables['path_to_relaxations']):
    print(folder)
    path_to_vasprun = f"{setup_variables['path_to_relaxations']}/{folder}/vasprun.xml.gz"
    
    # Read vasprun
    if os.path.exists(path_to_vasprun):
        # Read xml file with forces
        temp_vasprun = ase.io.read(path_to_vasprun)

        # Append to main list
        rattled_structures.append(temp_vasprun)

disp-0445
disp-0074
disp-0532
disp-0971
disp-0831
disp-0775
disp-0866
disp-0608
disp-0012
disp-0351
disp-0443
disp-0896
disp-0320
disp-0141
disp-0244
disp-0693
disp-0851
disp-0158
disp-0460
disp-0157
disp-0716
disp-1000
disp-0765
disp-0232
disp-0910
disp-0534
disp-0478
disp-0130
disp-0304
disp-0741
disp-0261
disp-0808
disp-0204
disp-0725
disp-0615
disp-0315
disp-0087
disp-0259
disp-0753
disp-0893
disp-0414
disp-0609
disp-0156
disp-0274
disp-0480
disp-0273
disp-0644
disp-0448
disp-0567
disp-0148
disp-0241
disp-0245
disp-0471
disp-0630
disp-0807
disp-0444
disp-0427
disp-0839
disp-0612
disp-0655
disp-0300
disp-0458
disp-0331
disp-0963
disp-0558
disp-0132
disp-0215
disp-0128
disp-0307
disp-0679
disp-0904
disp-0919
disp-0794
disp-0861
disp-0638
disp-0233
disp-0907
disp-0486
disp-0533
disp-0165
disp-0078
disp-0578
disp-0604
disp-0994
disp-0504
disp-0623
disp-0179
disp-0731
disp-0286
disp-0611
disp-0857
disp-0297
disp-0293
disp-0942
disp-0230
disp-0673
disp-0784
disp-0823
disp-0998
disp-0720


In [5]:
print(rattled_structures)

[Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=True, cell=[8.41723758, 8.45317928, 8.45317928], calculator=SinglePointDFTCalculator(...)), Atoms(symbols='Cr32N32', pbc=Tr

Set up cluster space

In [6]:
cs = ClusterSpace(primtive_cell, cutoffs)

# Save ClusterSpace
cs.write(setup_variables['path_to_cluster_space'])
print(cs)

Primitive cell:
    Formula: CrN
    Cell:
        [  2.10431   2.11329   0.00000]
        [  2.10431  -2.11329   0.00000]
        [ -2.10431   0.00000  -2.11329]
    Basis:
        Cr  [  0.50000   0.50000   0.00000]
        N   [  0.00000   0.00000   0.00000]

Crystal symmetry:
    Spacegroup:          I4/mmm (139)
    Unique site:         2
    Symmetry operations: 16
    symprec:             1.00e-05

Cutoffs:
    Maximum cutoff: 4.0
    Found 2 center atoms with 34 images totaling 36 atoms

Clusters:
    Clusters: {2: 53, 3: 352, 4: 535}
    Total number of clusters: 940

Orbits:
    Orbits: {2: 9, 3: 22, 4: 36}
    Total number of orbits: 67

Eigentensors:
    Eigentensors: {2: 28, 3: 220, 4: 728}
    Total number of parameters: 976
    Discarded orbits:
        (0, 0, 0)
        (1, 1, 1)

Constraints:
    Acoustic: True
    Number of degrees of freedom: {2: 24, 3: 183, 4: 409}
    Total number of degrees of freedom: 616
=================== Cluster Space ====================
Spa

In [7]:
cs.print_orbits()

===================================== List of Orbits =====================================
index | order |      elements      |  radius  |     prototype      | clusters | parameters
------------------------------------------------------------------------------------------
  0   |   2   |       Cr Cr        |  0.0000  |       (0, 0)       |    1     |     2     
  1   |   2   |        Cr N        |  1.0522  |       (0, 1)       |    2     |     2     
  2   |   2   |       Cr Cr        |  1.4912  |       (0, 2)       |    4     |     4     
  3   |   2   |        Cr N        |  1.8276  |       (0, 3)       |    8     |     5     
  4   |   2   |        Cr N        |  1.0566  |       (0, 5)       |    4     |     3     
  5   |   2   |       Cr Cr        |  1.4943  |       (0, 6)       |    2     |     3     
  6   |   2   |        N N         |  0.0000  |       (1, 1)       |    1     |     2     
  7   |   2   |        N N         |  1.4912  |       (1, 5)       |    4     |     4     

Set up structure container

In [8]:
structures = prepare_structures(rattled_structures, super_cell)
sc = StructureContainer(cs)
for structure in structures:
    sc.add_structure(structure)

# Save StructureContainer
sc.write(setup_variables['path_to_structure_container'])
print(sc)

=============== Structure Container ================
Total number of structures : 727
Number of force components : 139584
----------------------------------------------------
index | num-atoms | avg-disp | avg-force | max-force
----------------------------------------------------
 0    |    64     |  0.2986  |   2.6638  |   5.4121 
 1    |    64     |  0.2795  |   2.3795  |   5.1255 
 2    |    64     |  0.2849  |   2.3371  |   4.5165 
 3    |    64     |  0.2765  |   2.3122  |   4.3475 
 4    |    64     |  0.3030  |   2.0683  |   4.0355 
 5    |    64     |  0.2753  |   2.2454  |   4.5369 
 6    |    64     |  0.2900  |   2.2606  |   3.8235 
 7    |    64     |  0.2875  |   2.3280  |   5.0206 
 8    |    64     |  0.2818  |   2.5161  |   4.7912 
 9    |    64     |  0.2831  |   2.4536  |   4.3165 
 10   |    64     |  0.2944  |   2.2991  |   3.9870 
 11   |    64     |  0.2896  |   2.6814  |   5.7609 
 12   |    64     |  0.2874  |   2.5172  |   4.7525 
 13   |    64     |  0.2946  |

Train the model

In [9]:
opt = Optimizer(sc.get_fit_data())
opt.train()
print(opt)

===================== Optimizer ======================
seed                           : 42
fit_method                     : least-squares
standardize                    : True
n_target_values                : 139584
n_parameters                   : 616
n_nonzero_parameters           : 616
parameters_norm                : 8.771193
target_values_std              : 1.490554
rmse_train                     : 0.3423257
rmse_test                      : 0.3418501
R2_train                       : 0.9471957
R2_test                        : 0.9479236
AIC                            : -268106.1
BIC                            : -262105.6
train_size                     : 125625
test_size                      : 13959


Construct force constant potential

In [10]:
fcp = ForceConstantPotential(cs, opt.parameters)

# Save ForceConstantPotential
fcp.write(setup_variables['path_to_fcp'])
print(fcp)

=============== ForceConstantPotential ===============
Spacegroup I4/mmm (139)
Cell:
Cell([[2.1043093954868364, 2.113294820852644, 0.0], [2.1043093954868364, -2.113294820852644, 0.0], [-2.1043093954868364, 0.0, -2.113294820852644]])
Basis:
[[0.5 0.5 0. ]
 [0.  0.  0. ]]
Numbers: [24  7]
Cutoff matrix:
[[ 4.  4.  3.]
 [nan  4.  3.]
 [nan nan  3.]]
Order 2, #orbits 9, #cluster 28, #parameters 24
Order 3, #orbits 20, #cluster 140, #parameters 183
Order 4, #orbits 36, #cluster 202, #parameters 409
Total number of orbits: 65
total number of clusters: 370
total number of parameters: 616
